In [1]:
# Copyright (c) 2024 Microsoft Corporation.
# Licensed under the MIT License.

'\nCopyright (c) Microsoft Corporation.\n'

In [1]:
import os

import pandas as pd
import tiktoken

from graphrag.query.indexer_adapters import read_indexer_entities, read_indexer_reports
from graphrag.query.llm.oai.chat_openai import ChatOpenAI
from graphrag.query.llm.oai.typing import OpenaiApiType
from graphrag.query.structured_search.global_search.community_context import (
    GlobalCommunityContext,
)
from graphrag.query.structured_search.global_search.search import GlobalSearch

c:\Users\armoshar\AppData\Local\anaconda3\envs\gen-ai\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Global Search example

Global search method generates answers by searching over all AI-generated community reports in a map-reduce fashion. This is a resource-intensive method, but often gives good responses for questions that require an understanding of the dataset as a whole (e.g. What are the most significant values of the herbs mentioned in this notebook?).

### LLM setup

In [2]:
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("GRAPHRAG_API_KEY")
api_base = os.getenv("GRAPHRAG_API_BASE")
llm_model = os.getenv("GRAPHRAG_LLM_MODEL")
embedding_model = os.getenv("GRAPHRAG_EMBEDDING_MODEL")

llm = ChatOpenAI(
    api_key=api_key,
    api_version="2024-02-15-preview",
    api_base=api_base,
    model=llm_model,
    api_type=OpenaiApiType.AzureOpenAI,  # OpenaiApiType.OpenAI or OpenaiApiType.AzureOpenAI
    max_retries=20,
)
token_encoder = tiktoken.get_encoding("cl100k_base")

### Load community reports as context for global search

- Load all community reports in the `create_final_community_reports` table from the ire-indexing engine, to be used as context data for global search.
- Load entities from the `create_final_nodes` and `create_final_entities` tables from the ire-indexing engine, to be used for calculating community weights for context ranking. Note that this is optional (if no entities are provided, we will not calculate community weights and only use the `rank` attribute in the community reports table for context ranking)

In [3]:
# parquet files generated from indexing pipeline
INPUT_DIR = "./output/20240827-100424/artifacts"
COMMUNITY_REPORT_TABLE = "create_final_community_reports"
ENTITY_TABLE = "create_final_nodes"
ENTITY_EMBEDDING_TABLE = "create_final_entities"

# community level in the Leiden community hierarchy from which we will load the community reports
# higher value means we use reports from more fine-grained communities (at the cost of higher computation cost)
COMMUNITY_LEVEL = 2

In [4]:
entity_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_TABLE}.parquet")
report_df = pd.read_parquet(f"{INPUT_DIR}/{COMMUNITY_REPORT_TABLE}.parquet")
entity_embedding_df = pd.read_parquet(f"{INPUT_DIR}/{ENTITY_EMBEDDING_TABLE}.parquet")

reports = read_indexer_reports(report_df, entity_df, COMMUNITY_LEVEL)
entities = read_indexer_entities(entity_df, entity_embedding_df, COMMUNITY_LEVEL)
print(f"Total report count: {len(report_df)}")
print(
    f"Report count after filtering by community level {COMMUNITY_LEVEL}: {len(reports)}"
)
report_df.head()

c:\Users\armoshar\AppData\Local\anaconda3\envs\gen-ai\Lib\site-packages\graphrag\query\indexer_adapters.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].fillna(-1)
c:\Users\armoshar\AppData\Local\anaconda3\envs\gen-ai\Lib\site-packages\graphrag\query\indexer_adapters.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  entity_df["community"] = entity_df["community"].astype(int)


Total report count: 669
Report count after filtering by community level 2: 515


,community,full_content,level,rank,title,rank_explanation,summary,findings,full_content_json,id
0,649,# CFTC and FINRA Regulatory Community\n\nThe c...,3,8.0,CFTC and FINRA Regulatory Community,The impact severity rating is high due to the ...,The community is centered around the Commodity...,[{'explanation': 'The Commodity Futures Tradin...,"{\n ""title"": ""CFTC and FINRA Regulatory Com...",e93ce0f7-21bb-40fc-b2e6-f4bb4a60676f
1,650,# Securities Regulatory Community: SEC and CFT...,3,8.0,Securities Regulatory Community: SEC and CFTC,The impact severity rating is high due to the ...,The community centers around key regulatory en...,[{'explanation': 'The SEC (Securities and Exch...,"{\n ""title"": ""Securities Regulatory Communi...",a2c4aa3b-ecf4-469d-9f8f-748c1bbf6994
2,651,# FERC and Energy Regulation Community\n\nThe ...,3,8.5,FERC and Energy Regulation Community,The impact severity rating is high due to FERC...,The community is centered around the Federal E...,[{'explanation': 'The Federal Energy Regulator...,"{\n ""title"": ""FERC and Energy Regulation Co...",99e76ed5-1d7b-460f-a14e-94a382a5aa8c
3,652,"# Energy Regulatory Community: DOE, FERC, and ...",3,8.0,"Energy Regulatory Community: DOE, FERC, and NERC",The impact severity rating is high due to the ...,The community consists of key entities involve...,[{'explanation': 'The Department of Energy (DO...,"{\n ""title"": ""Energy Regulatory Community: ...",d238c245-30f8-4934-a085-a9b0f302302f
4,653,# Export-Import Bank and Economic Impact\n\nTh...,3,7.5,Export-Import Bank and Economic Impact,The impact severity rating is high due to the ...,The community centers around the Export-Import...,[{'explanation': 'The Export-Import Bank is an...,"{\n ""title"": ""Export-Import Bank and Econom...",ba5c57d3-f0c5-4618-8f31-274acded5921


#### Build global context based on community reports

In [5]:
context_builder = GlobalCommunityContext(
    community_reports=reports,
    entities=entities,  # default to None if you don't want to use community weights for ranking
    token_encoder=token_encoder,
)

#### Perform global search

In [6]:
context_builder_params = {
    "use_community_summary": False,  # False means using full community reports. True means using community short summaries.
    "shuffle_data": True,
    "include_community_rank": True,
    "min_community_rank": 0,
    "community_rank_name": "rank",
    "include_community_weight": True,
    "community_weight_name": "occurrence weight",
    "normalize_community_weight": True,
    "max_tokens": 12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    "context_name": "Reports",
}

map_llm_params = {
    "max_tokens": 1000,
    "temperature": 0.0,
    "response_format": {"type": "json_object"},
}

reduce_llm_params = {
    "max_tokens": 2000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 1000-1500)
    "temperature": 0.0,
}

In [7]:
search_engine = GlobalSearch(
    llm=llm,
    context_builder=context_builder,
    token_encoder=token_encoder,
    max_data_tokens=12_000,  # change this based on the token limit you have on your model (if you are using a model with 8k limit, a good setting could be 5000)
    map_llm_params=map_llm_params,
    reduce_llm_params=reduce_llm_params,
    allow_general_knowledge=False,  # set this to True will add instruction to encourage the LLM to incorporate general knowledge in the response, which may increase hallucinations, but could be useful in some use cases.
    json_mode=True,  # set this to False if your LLM model does not support JSON mode.
    context_builder_params=context_builder_params,
    concurrent_coroutines=32,
    response_type="multiple paragraphs",  # free form text describing the response type and format, can be anything, e.g. prioritized list, single paragraph, multiple paragraphs, multiple-page report
)

In [10]:
result = await search_engine.asearch(

    "How does the Project 2025 plan to empower women with gender equality what are the steps in the plan? "
)

print(result.response)

I am sorry but I am unable to answer this question given the provided data.


In [11]:
# inspect the data used to build the context for the LLM responses
result.context_data["reports"]

,id,title,occurrence weight,content,rank
0,4,Dulce Base and the Paranormal Military Squad: ...,1.000000,# Dulce Base and the Paranormal Military Squad...,8.5
1,5,Sam Rivera and the Paranormal Military Squad a...,1.000000,# Sam Rivera and the Paranormal Military Squad...,7.5
2,2,Dulce Base: Extraterrestrial Research and Comm...,0.750000,# Dulce Base: Extraterrestrial Research and Co...,8.5
3,3,Operation: Dulce and the Paranormal Military S...,0.583333,# Operation: Dulce and the Paranormal Military...,8.5
4,0,Dulce Base and the Paranormal Military Squad Team,0.416667,# Dulce Base and the Paranormal Military Squad...,8.5


In [12]:
# inspect number of LLM calls and tokens
print(f"LLM calls: {result.llm_calls}. LLM tokens: {result.prompt_tokens}")

LLM calls: 2. LLM tokens: 5231


: 